# Notebooks for testing 

## DataLoader

In [1]:
# -*- coding: utf-8 -*-
"""
data_loader.py

Optimized version: Implements lazy loading for better memory efficiency,
and uses context managers and error handling for robust database operations.
"""

import sqlite3
import pandas as pd


class DataLoader:
    """
    Class responsible for loading data from an SQLite database with lazy loading.
    """

    def __init__(self, db_path: str):
        """
        Args:
            db_path (str): Path to the SQLite database file.
        """
        self.db_path = db_path
        self._cache = {}

    def _connect(self):
        """
        Creates a connection to the SQLite database.

        Returns:
            sqlite3.Connection: Database connection object.
        """
        try:
            connection = sqlite3.connect(self.db_path)
            return connection
        except sqlite3.Error as e:
            print(f"Error connecting to database: {e}")
            raise

    def load_table(self, table_name: str) -> pd.DataFrame:
        """
        Lazily loads a table from the database into a Pandas DataFrame.

        Args:
            table_name (str): Name of the table to load.

        Returns:
            pd.DataFrame: DataFrame containing the table data.
        """
        if table_name in self._cache:
            return self._cache[table_name]

        try:
            with self._connect() as connection:
                query = f"SELECT * FROM {table_name}"
                df = pd.read_sql_query(query, connection)
            self._cache[table_name] = df  # Cache the table
            return df
        except sqlite3.Error as e:
            print(f"Error loading table '{table_name}': {e}")
            raise

    def get_available_tables(self) -> list:
        """
        Retrieves the list of available tables in the database.

        Returns:
            list: List of table names.
        """
        try:
            with self._connect() as connection:
                query = "SELECT name FROM sqlite_master WHERE type='table';"
                tables = [row[0] for row in connection.execute(query).fetchall()]
            return tables
        except sqlite3.Error as e:
            print(f"Error retrieving available tables: {e}")
            raise

    def clear_cache(self):
        """
        Clears the cached tables to free memory.
        """
        self._cache.clear()



db_path = "../data/F1_timingdata_2014_2019.sqlite"
data_loader = DataLoader(db_path)

try:
    print("Available Tables:", data_loader.get_available_tables())  # Lists available tables
    drivers_df = data_loader.load_table("drivers")  # Loads only "drivers" table
    print(drivers_df.head())  # Displays a preview
except Exception as e:
    print("An error occurred during data loading:", e)

Available Tables: ['drivers', 'starterfields', 'races', 'qualifyings', 'laps', 'retirements', 'fcyphases']
   id  carno initials              name
0   1     44      HAM    Lewis Hamilton
1   2      3      RIC  Daniel Ricciardo
2   3      6      ROS      Nico Rosberg
3   4     20      MAG   Kevin Magnussen
4   5     14      ALO   Fernando Alonso
